# Pandas



In [275]:
import pandas as pd

Firstly, reading the data.

In [276]:
pd_categories = pd.read_csv('categories.csv')

pd_cities = pd.read_csv('cities.csv')

pd_countries = pd.read_csv('countries.csv')

pd_customers = pd.read_csv('customers.csv')

pd_employees = pd.read_csv('employees.csv')

pd_products = pd.read_csv('products.csv')

pd_sales = pd.read_csv('sales.csv')

Now let's do filtering and drop NAs if there are any:

In [277]:
pd_categories = pd_categories.dropna()

pd_cities = pd_cities.dropna()

pd_countries = pd_countries.dropna()

pd_customers = pd_customers.dropna()

pd_employees = pd_employees.dropna()

pd_products = pd_products.dropna()

pd_sales = pd_sales.dropna()

And now I want to calculate properly TotalPrice of each sale and proceed to merging sales data and products data and defining the formula for TotalPrice.

In [278]:
pd_sales = pd_sales.merge(
    pd_products[['ProductID', 'Price']],
    on = 'ProductID',
    how = 'left'
)

In [279]:
pd_sales['TotalPrice'] = pd_sales['Price'] * pd_sales['Quantity']

I decided to drop uncessary now column Price:

In [280]:
pd_sales = pd_sales.drop(columns = ['Price'])

Now, I want to apply filtering and see all orders that were done in January 2018 and have more than 10 units ordered. Then sort the orders by their SalesDate.

In [281]:
pd_sales = pd_sales[pd_sales['Quantity'] > 10]

pd_sales = pd_sales[
    (pd_sales['SalesDate'] >= '2018-01-01') &
    (pd_sales['SalesDate'] <  '2018-04-01')
]

pd_sales = pd_sales.sort_values('SalesDate', ascending = False)

Here, let me calculate the amount a customer spends on all orders, the average value of the order, and the number of orders.

In [282]:
pd_aggregation = (
    pd_sales
    .groupby('CustomerID')
    .agg(
        total_spent = pd.NamedAgg('TotalPrice', 'sum'),
        avg_spent   = pd.NamedAgg('TotalPrice', 'mean'),
        order_count = pd.NamedAgg('SalesID', 'count')
      )
    .reset_index()
)

For convenience, I also rounded the amounts:

In [283]:
pd_aggregation['total_spent'] = round(pd_aggregation['total_spent'], 2)
pd_aggregation['avg_spent'] = round(pd_aggregation['avg_spent'], 2)

On this step I decided to classify customers into 3 groups: bronze, silver, and gold, according to the amounts they spent:

In [284]:
quantiles = pd_aggregation['total_spent'].quantile([0, 0.33, 0.66, 1.0])

pd_aggregation['customer_status'] = pd.cut(
    pd_aggregation['total_spent'],
    bins = quantiles.values,
    labels = ['bronze', 'silver', 'gold'],
    include_lowest = True
)

After that i decided to identify first and last orders of each customer:

In [285]:
first_orders = (
    pd_sales
    .groupby('CustomerID')['SalesDate']
    .min()
    .reset_index()
    .rename(columns={'SalesDate': 'first_order_date'})
)

In [287]:
pd_aggregation = pd_aggregation.merge(
    first_orders,
    on = 'CustomerID',
    how = 'left'
)

In [288]:
last_orders = (
    pd_sales
      .groupby('CustomerID')['SalesDate']
      .max()
      .reset_index()
      .rename(columns = {'SalesDate': 'last_order_date'})
)

In [289]:
pd_aggregation = pd_aggregation.merge(
    last_orders,
    on='CustomerID',
    how='left'
)

Last but not least, I decided to identify customer's favorite product - the one that was ordered the most frequently. I begin with identifying the id of such product, and then the name itself:

In [290]:
favorite_product_id = (
    pd_sales
      .groupby('CustomerID')['ProductID']
      .agg(lambda ids: ids.value_counts().idxmax())
      .reset_index(name = 'favorite_product_id')
)

In [291]:
favorite_product_name = (
    favorite_product_id
    .merge(
        pd_products[['ProductID','ProductName']],
        left_on = 'favorite_product_id',
        right_on = 'ProductID',
        how = 'left'
      )
    .drop(columns = ['ProductID','favorite_product_id'])
    .rename(columns = {'ProductName':'favorite_product'})
)

In [292]:
pd_aggregation = pd_aggregation.merge(
    favorite_product_name,
    on = 'CustomerID',
    how='left'
)

Tadaaa! Final table looks like:

In [293]:
pd_aggregation

,CustomerID,total_spent,avg_spent,order_count,customer_status,first_order_date,last_order_date,favorite_product
0,39504,11400.53,670.62,17,bronze,2018-01-04 03:09:41.690,2018-03-27 14:45:14.350,Peas - Frozen
1,39505,27411.50,559.42,49,bronze,2018-01-03 15:00:24.840,2018-03-27 15:45:05.140,"Pork - Loin, Bone - In"
2,39506,20865.45,632.29,33,bronze,2018-01-02 23:04:14.980,2018-03-30 17:13:40.410,Pastry - Cheese Baked Scones
3,39507,23219.67,494.04,47,bronze,2018-01-03 00:08:23.260,2018-03-29 23:50:53.470,Sauce - Demi Glace
4,39508,30336.88,551.58,55,bronze,2018-01-04 04:06:00.020,2018-03-27 03:09:35.830,Veal - Sweetbread
...,...,...,...,...,...,...,...,...
59251,98755,58086.71,1117.05,52,gold,2018-01-08 01:17:18.300,2018-03-31 10:08:55.370,Cattail Hearts
59252,98756,75462.62,1197.82,63,gold,2018-01-01 20:37:26.440,2018-03-30 13:40:40.650,Juice - Happy Planet
59253,98757,66226.58,1379.72,48,gold,2018-01-02 10:21:56.120,2018-03-29 17:45:08.070,"Appetizer - Mini Egg Roll, Shrimp"
59254,98758,60422.53,1258.80,48,gold,2018-01-02 04:22:05.810,2018-03-31 15:38:28.190,"Yogurt - Blueberry, 175 Gr"


Finally, saving the output file:

In [294]:
pd_aggregation.to_csv("pandas_output.csv", index = False)

# Polars

In [295]:
import polars as pl
import typing
import polars._typing as _t
from datetime import datetime

Firstly, reading the data (again but with Polars).

In [296]:
pl_categories = pl.read_csv("categories.csv").drop_nulls()
pl_cities = pl.read_csv("cities.csv").drop_nulls()
pl_countries = pl.read_csv("countries.csv").drop_nulls()
pl_customers = pl.read_csv("customers.csv").drop_nulls()
pl_employees = pl.read_csv("employees.csv").drop_nulls()
pl_products = pl.read_csv("products.csv").drop_nulls()
pl_sales = pl.read_csv("sales.csv").drop_nulls()

Here I'm joining the sales table with prices, to compute TotalPrice = Price * Quantity and then drop Price from merged data:

In [297]:
pl_sales = pl_sales.join(
    pl_products.select(['ProductID', 'Price']),
    on = 'ProductID',
    how = 'left'
)

In [298]:
pl_sales = (
    pl_sales
    .with_columns(
        (pl.col('Price') * pl.col('Quantity')).alias('TotalPrice')
    )
    .drop('Price')
)

Now I proceed to filter order with quantity over 10 units and placed within 1 January and 1 April, sorting them by date.

In [299]:
pl_sales = pl_sales.filter(pl.col('Quantity') > 10)

In [300]:
pl_sales = (
    pl_sales.with_columns(
        pl.col('SalesDate')
        .str.to_datetime('%Y-%m-%d %H:%M:%S.%f', strict = False)
        .dt.date()
        .alias('SalesDate')
    )
    .filter(
        (pl.col('SalesDate') >= date(2018, 1, 1)) &
        (pl.col('SalesDate') <  date(2018, 4, 1))
    )
)

<ipython-input-300-5dc128cd89f6>:4: ChronoFormatWarning: Detected the pattern `.%f` in the chrono format string. This pattern should not be used to parse values after a decimal point. Use `%.f` instead. See the full specification: https://docs.rs/chrono/latest/chrono/format/strftime
  .str.to_datetime('%Y-%m-%d %H:%M:%S.%f', strict = False)


In [301]:
pl_sales = pl_sales.sort('SalesDate', descending = True)

Here, let me calculate the amount a customer spends on all orders, the average value of the order, and the number of orders.

In [311]:
pl_aggregation = pl_sales.group_by('CustomerID').agg([
    pl.col('TotalPrice').sum().round(2).alias('total_spent'),
    pl.col('TotalPrice').mean().round(2).alias('avg_spent'),
    pl.count('SalesID').alias('order_count'),
])

Computing quantiles to classify the customers:

In [312]:
q33 = pl_aggregation['total_spent'].quantile(0.33)
q66 = pl_aggregation['total_spent'].quantile(0.66)

In [313]:
pl_aggregation = pl_aggregation.with_columns([
    pl.when(pl.col('total_spent') <= q33).then(pl.lit('bronze'))
      .when(pl.col('total_spent') <= q66).then(pl.lit('silver'))
      .otherwise(pl.lit('gold'))
      .alias('customer_status')
])

Then let me identify first and last order of the customer:


In [314]:
first_orders = pl_sales.group_by('CustomerID').agg(
    pl.col('SalesDate').min().alias('first_order_date')
)

last_orders = pl_sales.group_by('CustomerID').agg(
    pl.col('SalesDate').max().alias('last_order_date')
)

pl_aggregation = (
    pl_aggregation
    .join(first_orders, on = 'CustomerID', how = 'left')
    .join(last_orders,  on = 'CustomerID', how = 'left')
)

Last but not least, I decided to identify customer's favorite product - the one that was ordered the most frequently. I begin with identifying the id of such product, and then the name itself:

In [315]:
product_counts = pl_sales.group_by(['CustomerID', 'ProductID']).agg(
    pl.count('ProductID').alias('count')
)

In [316]:
favorite_product_id = (
    product_counts
      .sort(['CustomerID', 'count'], descending = [False, True])
      .group_by('CustomerID')
      .agg(pl.first('ProductID').alias('favorite_product_id'))
)

In [317]:
favorite_product_name = (
    favorite_product_id
    .join(
        pl_products.select(['ProductID', 'ProductName']),
        left_on = 'favorite_product_id',
        right_on = 'ProductID',
        how = 'left'
    )
    .select([
        'CustomerID',
        pl.col('ProductName').alias('favorite_product')
    ])
)

In [318]:
pl_aggregation = pl_aggregation.join(favorite_product_name, on = 'CustomerID', how = 'left')

Final output here:

In [319]:
pl_aggregation

CustomerID,total_spent,avg_spent,order_count,customer_status,first_order_date,last_order_date,favorite_product
i64,f64,f64,u32,str,date,date,str
79312,38326.05,982.72,39,"""silver""",2018-01-04,2018-03-29,"""Ice Cream Bar - Hageen Daz To"""
45446,27157.56,662.38,41,"""bronze""",2018-01-04,2018-03-31,"""Peas - Frozen"""
62330,38155.56,794.91,48,"""silver""",2018-01-02,2018-03-30,"""Sage - Ground"""
82084,51534.92,1120.32,46,"""gold""",2018-01-01,2018-03-29,"""Ketchup - Tomato"""
75377,49489.37,951.72,52,"""gold""",2018-01-02,2018-03-30,"""Foam Dinner Plate"""
…,…,…,…,…,…,…,…
89872,53538.79,1029.59,52,"""gold""",2018-01-02,2018-03-31,"""Yogurt - French Vanilla"""
44179,23293.96,541.72,43,"""bronze""",2018-01-03,2018-03-30,"""Juice - V8 Splash"""
65637,38692.55,879.38,44,"""silver""",2018-01-05,2018-03-28,"""Wasabi Powder"""


Finally, saving the output:

In [ ]:
pl_aggregation.write_csv("polars_output.csv")